In [13]:
from pathlib import Path
import datetime as dt
import sys
import os
import tempfile

import googlemaps
import pandas as pd
import geopandas as gpd

sys.path.append('../')

import googlemaps_helpers as gh

ROOT = Path('../')
DATA_DIR = ROOT/'data'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Initialize the Google Maps API client with your API key
my_key = os.environ['ALEX_GOOGLE_KEY']
client = googlemaps.Client(key=my_key)

# Load points
path = DATA_DIR/'points.geojson'
points = gpd.read_file(str(path))
points


,id,geometry
0,a,POINT (174.7183227539062 -36.78179217131925)
1,b,POINT (174.7567749023438 -36.89829270211866)
2,c,POINT (174.9174499511719 -36.92135192790114)


In [4]:
# Call the Google Maps Distance Matrix API on some sample points
origs = [p.coords[0] for p in points['geometry']]
orig_ids = points.id
dests = [p.coords[0] for p in points['geometry']]
dest_ids = points.id
time = dt.datetime(2020, 5, 17, 8)
 
r = client.distance_matrix(gh.flip_coords(origs), gh.flip_coords(dests), mode='driving', 
  traffic_model='pessimistic', departure_time=time)
r


{'destination_addresses': ['9 Waverley Ave, Glenfield, Auckland 0629, New Zealand',
  '919 Mount Eden Rd, Mount Eden, Auckland 1024, New Zealand',
  '26 Ben Nevis Pl, Northpark, Auckland 2013, New Zealand'],
 'origin_addresses': ['9 Waverley Ave, Glenfield, Auckland 0629, New Zealand',
  '919 Mount Eden Rd, Mount Eden, Auckland 1024, New Zealand',
  '26 Ben Nevis Pl, Northpark, Auckland 2013, New Zealand'],
 'rows': [{'elements': [{'distance': {'text': '1 m', 'value': 0},
     'duration': {'text': '1 min', 'value': 0},
     'duration_in_traffic': {'text': '1 min', 'value': 0},
     'status': 'OK'},
    {'distance': {'text': '20.4 km', 'value': 20350},
     'duration': {'text': '26 mins', 'value': 1535},
     'duration_in_traffic': {'text': '25 mins', 'value': 1522},
     'status': 'OK'},
    {'distance': {'text': '33.2 km', 'value': 33157},
     'duration': {'text': '38 mins', 'value': 2299},
     'duration_in_traffic': {'text': '39 mins', 'value': 2314},
     'status': 'OK'}]},
  {'el

In [5]:
# Build a distance matrix between all pairs of points
print(gh.compute_cost(points.shape[0]**2))

gh.build_distance_matrix_df(client, points, points, origin_id_col='id',
  destination_id_col='id')



#elements                                     9
exceeds 100000-element daily limit?       False
estimated cost for job in USD            0.0045
estimated duration for job in minutes    0.0015
dtype: object


,origin_address,destination_address,origin_id,destination_id,duration,distance
0,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",a,a,0,0
1,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",a,b,1535,20350
2,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",a,c,2299,33157
3,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",b,a,1512,18226
4,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",b,b,0,0
5,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",b,c,1754,20528
6,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",c,a,2286,31168
7,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",c,b,1660,18567
8,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",c,c,0,0


In [16]:
with tempfile.TemporaryDirectory() as out_dir:
    gh.run_distance_matrix_job(client, points, points, out_dir=out_dir, mode='transit')

    %ls {out_dir}
    f = pd.read_csv(out_dir + '/transit_from_orig_row_0_to_dest_row_0--dest_row_2.csv')

f

2018-05-15 10:14:09,854 root         INFO     
Working on origin 1 of 3 (id orig_row_0)
2018-05-15 10:14:09,854 root         INFO     
Working on origin 1 of 3 (id orig_row_0)
2018-05-15 10:14:10,705 root         INFO     
Working on origin 2 of 3 (id orig_row_1)
2018-05-15 10:14:10,705 root         INFO     
Working on origin 2 of 3 (id orig_row_1)
2018-05-15 10:14:11,588 root         INFO     
Working on origin 3 of 3 (id orig_row_2)
2018-05-15 10:14:11,588 root         INFO     
Working on origin 3 of 3 (id orig_row_2)


transit_from_orig_row_0_to_dest_row_0--dest_row_2.csv
transit_from_orig_row_1_to_dest_row_0--dest_row_2.csv
transit_from_orig_row_2_to_dest_row_0--dest_row_2.csv


,origin_address,destination_address,origin_id,destination_id,duration,distance
0,"12 Waverley Ave, Glenfield, Auckland 0629, New...","12 Waverley Ave, Glenfield, Auckland 0629, New...",orig_row_0,dest_row_0,0,0
1,"12 Waverley Ave, Glenfield, Auckland 0629, New...","917 Mount Eden Rd, Mount Eden, Auckland 1024, ...",orig_row_0,dest_row_1,4556,20066
2,"12 Waverley Ave, Glenfield, Auckland 0629, New...","23 Ben Nevis Pl, Northpark, Auckland 2013, New...",orig_row_0,dest_row_2,6592,38669
